## 1. Load NERs extracted previously

In [ ]:
import json

with open("../out/prompting_ners_gpt_4.1_mini.json", "r", encoding="utf-8") as fr:
    prompting_ners = json.load(fr)

## 2. Preparing LLM for RE

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
with open("../data/introduccion.txt", "r", encoding="utf-8") as fr:
    text = fr.read()
    
with open("../data/historia_juan_rana.txt", "r", encoding="utf-8") as fr:
    text += fr.read()

### 2.1. First approach: Analyze the entire text passing NERs and text into the prompt

In [11]:
from langchain.prompts import PromptTemplate

entire_text_prompt = PromptTemplate(
    input_variables=["text", "entities"],
    template="""
        Dado el siguiente texto y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre dichas entidades.

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Lista de entidades (NERs):
        {entities}

        Texto de entrada:
        {text}

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
    """,
)

In [17]:
response = llm.invoke(
    entire_text_prompt.invoke({
        "text": text,
        "entities": "\n".join(
            [
                f"{key}: {', '.join(values)}"
                for key, values in prompting_ners.items()
            ]
        ),
    })
).content

response

"(Cosme Pérez)-[fue conocido artísticamente como]-(Juan Rana)  \n(Cosme Pérez)-[fue hijo de]-(Damián Pérez)  \n(Cosme Pérez)-[fue hijo de]-(Isabel de Basto)  \n(Cosme Pérez)-[estuvo casado con]-(María de Acosta)  \n(Cosme Pérez)-[tuvo hija]-(Francisca María Pérez)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Tomás Fernández)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Juan Bautista Valenciano)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Pedro de la Rosa)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Antonio de Prado)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Antonio García de Prado)  \n(Cosme Pérez)-[formaba parte de]-(compañía de Luis López Sustaete)  \n(Cosme Pérez)-[formó parte de]-(compañía de Diego Osorio)  \n(Cosme Pérez)-[formó parte de]-(compañía de Sebastián de Prado)  \n(Cosme Pérez)-[actuó en obra de]-(Luis de Belmonte)  \n(Cosme Pérez)-[actuó en obra de]-(Luis Quiñones de Benavente)  \n(Cosme Pérez)-[actuó en obra de]-(Antonio de Solís)  \n(Cosme Pér

In [18]:
relations = set()

for relation in response.split("\n"):
    print(relation.strip())
    relations.add(relation.strip())

(Cosme Pérez)-[fue conocido artísticamente como]-(Juan Rana)
(Cosme Pérez)-[fue hijo de]-(Damián Pérez)
(Cosme Pérez)-[fue hijo de]-(Isabel de Basto)
(Cosme Pérez)-[estuvo casado con]-(María de Acosta)
(Cosme Pérez)-[tuvo hija]-(Francisca María Pérez)
(Cosme Pérez)-[formaba parte de]-(compañía de Tomás Fernández)
(Cosme Pérez)-[formaba parte de]-(compañía de Juan Bautista Valenciano)
(Cosme Pérez)-[formaba parte de]-(compañía de Pedro de la Rosa)
(Cosme Pérez)-[formaba parte de]-(compañía de Antonio de Prado)
(Cosme Pérez)-[formaba parte de]-(compañía de Antonio García de Prado)
(Cosme Pérez)-[formaba parte de]-(compañía de Luis López Sustaete)
(Cosme Pérez)-[formó parte de]-(compañía de Diego Osorio)
(Cosme Pérez)-[formó parte de]-(compañía de Sebastián de Prado)
(Cosme Pérez)-[actuó en obra de]-(Luis de Belmonte)
(Cosme Pérez)-[actuó en obra de]-(Luis Quiñones de Benavente)
(Cosme Pérez)-[actuó en obra de]-(Antonio de Solís)
(Cosme Pérez)-[actuó en obra de]-(Agustín Moreto)
(Cosme Pé

In [22]:
print(f"Total de relaciones encontradas: {len(response.split("\n"))}, de las cuales {len(relations)} son únicas.")

Total de relaciones encontradas: 1669, de las cuales 63 son únicas.


In [29]:
for rel in relations:
    if len(rel.split("-")) >= 3:
        print(rel)

(Cosme Pérez)-[formaba parte de]-(compañía de Juan Bautista Valenciano)
(Cosme Pérez)-[actuó en obra de]-(Luis de Belmonte)
(Cosme Pérez)-[actuó en obra de]-(Antonio de Prado)
(Cosme Pérez)-[estuvo casado con]-(María de Acosta)
(Cosme Pérez)-[actuó en obra de]-(Juan Araña)
(Cosme Pérez)-[actuó en obra de]-(Luis Quiñones de Benavente)
(Cosme Pérez)-[actuó en obra de]-(Pedro de Cifuentes)
(Cosme Pérez)-[actuó en obra de]-(María de Quiñones)
(Cosme Pérez)-[actuó en obra de]-(Juan Ramos)
(Cosme Pérez)-[actuó en obra de]-(Juan Pérez de Montalbán)
(Cosme Pérez)-[fue hijo de]-(Damián Pérez)
(Cosme Pérez)-[actuó en obra de]-(Antonio de Solís)
(Cosme Pérez)-[formó parte de]-(compañía de Sebastián de Prado)
(Cosme Pérez)-[actuó en obra de]-(Mateo de Godoy)
(Cosme Pérez)-[actuó en obra de]-(Antonio Mejía)
(Cosme Pérez)-[actuó en obra de]-(Diego Osorio)
(Cosme Pérez)-[actuó en obra de]-(Antonio de Escamilla)
(Cosme Pérez)-[actuó en obra de]-(Agustín Moreto)
(Cosme Pérez)-[actuó en obra de]-(Juan V

In [30]:
with open("../out/full_text_re.txt", "w", encoding="utf-8") as fw:
    unique_relations = '\n'.join(rel for rel in relations if len(rel.strip().split("-")) >= 3)
    fw.write(unique_relations)

### 2.2. Second approach: Analyze each paragraph

In [31]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [32]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [33]:
with open("../data/introduccion.txt", "r", encoding="utf-8") as fr:
    text = fr.read()
    
with open("../data/historia_juan_rana.txt", "r", encoding="utf-8") as fr:
    text += fr.read()

In [34]:
from langchain.prompts import PromptTemplate

entire_text_prompt = PromptTemplate(
    input_variables=["text", "entities"],
    template="""
        Dado el siguiente texto y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre dichas entidades.

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
        Lista de entidades (NERs):
        {entities}

        Texto de entrada:
        {text}
    """,
)

In [38]:
from tqdm import tqdm

entities = "\n".join([
    f"{key}: {', '.join(values)}"
    for key, values in prompting_ners.items()
])

total_re_found = 0
relations = set()

for paragraph in tqdm(text.split("\n\n")):
    if not paragraph.strip():
        continue

    response = llm.invoke(
        entire_text_prompt.invoke({
            "text": paragraph,
            "entities": entities,
        })
    ).content
    
    total_re_found += len(response.split("\n"))

    for relation in response.split("\n"):
        relation = relation.strip()
        if relation and len(relation.split("-")) >= 3:
            relations.add(relation)

100%|██████████| 37/37 [03:03<00:00,  4.96s/it]


In [40]:
print(f"Total de relaciones encontradas: {total_re_found}, de las cuales {len(relations)} son únicas.")

Total de relaciones encontradas: 497, de las cuales 485 son únicas.


In [41]:
with open("../out/per_paragraph_text_re.txt", "w", encoding="utf-8") as fw:
    unique_relations = '\n'.join(rel for rel in relations if len(rel.strip().split("-")) >= 3)
    fw.write(unique_relations)

### 2.3. Third approach: Analyze each sentence with overlap

In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
import os

In [11]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [42]:
import json

with open("../out/prompting_ners_gpt_4.1_mini.json", "r", encoding="utf-8") as fr:
    prompting_ners = json.load(fr)

In [9]:
with open("../data/introduccion.txt", "r", encoding="utf-8") as fr:
    text = fr.read()
    
with open("../data/historia_juan_rana.txt", "r", encoding="utf-8") as fr:
    text += fr.read()

In [61]:
import ast
    
data = pd.read_csv("../data/DicatJuanRana_w_clarified_sentences.csv", sep=";", encoding="utf-8", converters={"sentences": ast.literal_eval, "clarified_sentences": ast.literal_eval})

In [ ]:
from langchain.prompts import PromptTemplate

entire_text_prompt = PromptTemplate(
    input_variables=["text", "context", "entities"],
    template="""
        Dada la siguiente frase y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre dichas entidades.
        Las frases pueden contener un contexto para ayudar a identificar las relaciones (para frases que comiencen con fue..., el cual..., etc.) pero no se deben sacar relaciones del propio contexto.

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
        Lista de entidades (NERs):
        {entities}

        Contexto de la frase:
        Cosme Pérez. 
        {context}

        Frase de entrada:
        {text}
    """,
)

In [66]:
from tqdm.auto import tqdm
from collections import deque

entities = "\n".join([
    f"{key}: {', '.join(values)}"
    for key, values in prompting_ners.items()
    if key != "DATE"  # Exclude DATE entities for this task
])

sentences_of_context = 2 # Number of sentences to consider as context for each sentence

total_re_found = 0
relations = set()

for _, year in tqdm(data[data["year"].astype(int) == 1653].iterrows(), position=0):
    if not year["text"].strip():
        continue
        
    context = deque([])
    
    for sentence in tqdm(year["clarified_sentences"], position=1, leave=False):
        if not sentence.strip():
            continue

        response = llm.invoke(
            entire_text_prompt.invoke({
                "text": sentence.strip(),
                "context": "\n".join(context),
                "entities": entities,
            })
        ).content
    
        total_re_found += len(response.split("\n"))

        for relation in response.split("\n"):
            relation = relation.strip()
            if relation and len(relation.split("-")) >= 3:
                relations.add(relation)
        
        context.append(sentence.strip())
        if len(context) > sentences_of_context:
            context.popleft()

0it [00:00, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
# from tqdm.auto import tqdm
# from collections import deque

# entities = "\n".join([
#     f"{key}: {', '.join(values)}"
#     for key, values in prompting_ners.items()
# ])

# sentences_of_context = 2 # Number of sentences to consider as context for each sentence
# total_sentences = sum(len(paragraph.split(".")) for paragraph in text.split("\n\n") if paragraph.strip())

# total_re_found = 0
# relations = set()

# for paragraph in tqdm(text.split("\n\n"), position=0):
#     if not paragraph.strip():
#         continue
        
#     context = deque([])
    
#     for sentence in tqdm(paragraph.split("."), position=1, leave=False):
#         if not sentence.strip():
#             continue

#         response = llm.invoke(
#             entire_text_prompt.invoke({
#                 "text": sentence.strip(),
#                 "context": "\n".join(context),
#                 "entities": entities,
#             })
#         ).content
    
#         total_re_found += len(response.split("\n"))

#         for relation in response.split("\n"):
#             relation = relation.strip()
#             if relation and len(relation.split("-")) >= 3:
#                 relations.add(relation)
        
#         context.append(sentence.strip())
#         if len(context) > sentences_of_context:
#             context.popleft()

In [67]:
print(f"Total de relaciones encontradas: {total_re_found}, de las cuales {len(relations)} son únicas.")

Total de relaciones encontradas: 14, de las cuales 14 son únicas.


In [60]:
import rich as rc

rc.print(data[data["year"].astype(int) == 1653]["clarified_sentences"].values[0])

[
    'En una carta, fechada en Madrid el 9 de junio de este año, Felipe IV daba cuenta a doña Luisa Enríquez 
Manrique de la comedia Andrómeda y Perseo, de Calderón, representada en el Coliseo del Buen Retiro en la segunda 
quincena de mayo:',
    "Muy buena ha sido la comedia Andrómeda y Perseo, y vuestro amigo 'Juan Rana' ha cumplido famosamente con sus 
obligaciones;",
    'Bien, creo que si hubierais visto a Cosme Pérez, alias Juan Rana, se atrevería la risa a inquietar la 
función.',
    'La gente moza se ha entretenido harto" —la comedia representada Andrómeda y Perseo, de Calderón, fue Andrómeda
y Perseo, en la que Cosme Pérez representó el papel de "Bato"—;',
    'entre la documentación relativa a las fiestas del Corpus de Madrid consta una lista de los gastos de las 
fiestas del Corpus de este año, en la que se indica que Cosme Pérez recibió 1.000 rs.;',
    'Consta la respuesta, transmitida el 22 de noviembre por el Conde de Altamira, a la solicitud de "Cosme Pérez, 
alias Juan Rana" realizada el 18 de dicho mes, referente al "paso de una ración ordinaria que goza por la despensa 
de la casa de la Reyna nuestra señora, para una hija de la Reyna nuestra señora [Francisca María Pérez], con 
calidad que sea desde luego y que la susodicha no la pueda gozar si hubiese de andar en la farsa".'
]

In [68]:
relations

{'(Andrómeda y Perseo)-[es obra de]-(Calderón de la Barca)',
 '(Andrómeda y Perseo)-[es obra de]-(Calderón)',
 '(Andrómeda y Perseo)-[se representó en]-(Coliseo del Buen Retiro)',
 '(Conde de Altamira)-[transmitió respuesta a]-(Cosme Pérez)',
 '(Cosme Pérez)-[alias]-(Juan Rana)',
 '(Cosme Pérez)-[es alias de]-(Juan Rana)',
 '(Cosme Pérez)-[recibe]-(paso de una ración ordinaria)',
 '(Cosme Pérez)-[recibió]-(1.000 rs.)',
 '(Cosme Pérez)-[representó el papel de]-(Bato)',
 '(Cosme Pérez)-[tiene hija]-(Francisca María Pérez)',
 '(Felipe IV)-[daba cuenta a]-(Luisa Enríquez Manrique)',
 '(Francisca María Pérez)-[no puede gozar]-(paso de una ración ordinaria)',
 '(Juan Rana)-[ha cumplido con sus obligaciones]-(Andrómeda y Perseo)',
 '(paso de una ración ordinaria)-[goza por]-(casa de la Reyna nuestra señora)'}

In [13]:
with open("../out/per_sentence_text_re.txt", "w", encoding="utf-8") as fw:
    unique_relations = '\n'.join(rel for rel in relations if len(rel.strip().split("-")) >= 3)
    fw.write(unique_relations)